In [ ]:
from ase.io import read
from hubbard.data import AtomicData
from hubbard.gutz import GhostGutzwiller
from hubbard.utils.tools import setup_seed
from hubbard.utils.make_kpoints import kmesh_sampling
from hubbard.data import block_to_feature
from hubbard.utils.tools import get_semicircle_e_list
import numpy as np
from hubbard.data import _keys

setup_seed(1234)
U = 1.
J = 0.05 * U
Up = U - 2*J
Jp = J

alpha = 1.
Delta = 0.25
e_list = get_semicircle_e_list(nmesh=1000)
eks = alpha * e_list[:,None,None] * np.eye(5)[None,:,:]

onsite = np.eye(5) * Delta
onsite[0,0] = 0.
onsite[1,1] = 0.
onsite = onsite[None,:,:]
phy_onsite = {
    "C": onsite
}

intparams = {"C":[{"U":U,"Up":Up,"J":J, "Jp":Jp}]}


gga = GhostGutzwiller(
    atomic_number=np.array([6]),
    nocc=6,
    basis={"C":[5]},
    idx_intorb={"C":[0]},
    naux=1,
    intparams=intparams,
    nspin=1,
    kBT=0.002,
    mutol=1e-4,
    natural_orbital=True,
    decouple_bath=False,
    solver="DMRG",
    mixer_options={"method": "PDIIS", "a": 0.5, "k": 1},
    iscomplex=False,
    solver_options={"reorder": True, "n_threads": 15, "nupdate": 8, "eig_cutoff": 1e-7, "bond_dim": 1500, "su2": False, "iprint": 1, "scratch_dir": "./tmp_dmrg"},
)

atomicdata = AtomicData.from_ase(
    read("./gGA/test/C_cube.vasp"),
    r_max=3.1
    )

atomicdata = AtomicData.to_AtomicDataDict(atomicdata)
atomicdata[_keys.HAMILTONIAN_KEY] = eks
atomicdata[_keys.PHY_ONSITE_KEY] = phy_onsite

driver threads:   OpenMP = 1 TBB = 0 BLIS = 0 MKL = NONE  SeqType = Tasked MKLIntLen = 4
 THREADING = 2 layers : Global | Operator BatchedGEMM 
 NUMBER : Global = 15 Operator = 15 Quanta = 0 MKL = 1
 COMPLEX = 1 SINGLE-PREC = 0 KSYMM = 0


In [2]:
gga.run(atomicdata, 200, 1e-4, ckptprefix="h5odmrg")

DM_kin:  [0.15349909 0.15349909 0.37542716 0.37542716 0.50007125 0.50007125
 0.97251639 0.97251639 0.99844381 0.99844381]
[[-6.65770300e-10 -9.09709058e-10 -1.07210886e-10 -1.07318918e-10
  -5.23236870e-11 -3.47943924e-10  2.44727266e-10  1.92878864e-10
  -5.51475254e-11  2.52138296e-10]
 [-9.09709058e-10 -1.26830707e-09 -2.75713272e-11 -4.69734529e-11
  -4.88513952e-11 -3.05089037e-10  4.31746042e-10  1.59590129e-10
  -1.83551924e-10  2.63564025e-10]
 [-1.07210886e-10 -2.75713272e-11 -4.80515849e-10 -1.99723266e-10
   3.38243281e-10 -6.72617517e-11 -2.44444937e-10 -1.62939773e-10
   5.08645653e-10  1.48040288e-10]
 [-1.07318918e-10 -4.69734529e-11 -1.99723266e-10  4.77345385e-11
   3.72896575e-10  3.84612286e-10 -3.57171015e-10 -1.29864217e-10
   4.09491739e-10 -7.67152331e-11]
 [-5.23236870e-11 -4.88513952e-11  3.38243281e-10  3.72896575e-10
   1.54972452e-10  8.04922129e-10 -4.05427608e-10  1.17680990e-10
  -4.82075491e-12 -3.29146849e-10]
 [-3.47943924e-10 -3.05089037e-10 -6.726175

/homes/nessa/zhanghao/dev/Hubbard/gGA/nao/lanczos.py:42: UserWarning: loss of orthogonality @icgs.
  warnings.warn('loss of orthogonality @icgs.')



Sweep =    0 | Direction =  forward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      0.964 | E =      -0.3407807642 | DW = 1.70329e-06

Sweep =    1 | Direction = backward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      1.123 | E =      -0.3410230794 | DE = -2.42e-04 | DW = 1.58379e-06

Sweep =    2 | Direction =  forward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      1.292 | E =      -0.3410314775 | DE = -8.40e-06 | DW = 1.45785e-06

Sweep =    3 | Direction = backward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      1.451 | E =      -0.3410295050 | DE = 1.97e-06 | DW = 1.26791e-06

Sweep =    4 | Direction =  forward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      1.621 | E =      -0.3410295951 | DE = -9.01e-08 | DW = 1.29128e-06

Sweep =    5 | Direction = backward |

AssertionError: spin symmetry breaking error 0.9938880458936515

In [7]:
np.linalg.eigvalsh(gga.RDM["C"][0])
print(gga.gGAtomic.S2)

{'C': array([[2.07962856]])}


QH SU2: array([0.54767732, 0.54767732, 0.54795683, 0.54795683, 0.55593804,
       0.55593804, 0.67414707, 0.67414707, 0.67428104, 0.67428104])

SGF: array([0.51778631, 0.51779986, 0.51779994, 0.51780276, 0.51781543,
       0.5178351 , 0.72141419, 0.72141427, 0.72141495, 0.72142095])

QH SZ: array([0.54765287, 0.54765287, 0.54790089, 0.54790089, 0.55592029,
       0.55592029, 0.67419743, 0.67419743, 0.67432851, 0.67432851])

In [3]:
from gGA.operator import Slater_Kanamori, Operator
import numpy as np

N = 4
U = 1
J = 0.25 * U
Up = U-2*J
Jp = J

BD = 1500

# T = gga.gGAtomic.interact_ansatz[0].singleOrbs[0].solver._t
T = np.random.randn(N,N)
T = T + T.T
T = np.kron(T, np.eye(2))
sop = Slater_Kanamori(nsites=N, t=T, U=U, Up=Up, J=J, Jp=Jp, n_noninteracting=1)

In [4]:
from pyblock2.driver.core import DMRGDriver, SymmetryTypes

h1e, g2e = sop.get_Hqc(nsites=N)
ofd = 0.5 * (g2e[:N,:N,N:,N:] + g2e[N:,N:,:N,:N])
g2e[:N,:N,N:,N:] = ofd.copy()
g2e[N:,N:,:N,:N] = ofd.copy()
sop_re = sop.from_Hqc(h1e, g2e)
driver = DMRGDriver(
    scratch="/tmp/", 
    symm_type=SymmetryTypes.SZ, 
    n_threads=10,
    stack_mem=5368709120
    )
driver.initialize_system(n_sites=N, n_elec=N, spin=0)
Hemb = driver.get_qc_mpo(
                h1e=[h1e[:N, :N], h1e[N:,N:]], 
                g2e=[g2e[:N, :N, :N, :N], g2e[:N, :N, N:, N:], g2e[N:, N:, N:, N:]], 
                ecore=0., 
                iprint=0,
                reorder="gaopt"
                )

# Hemb = driver.get_qc_mpo(
#                 h1e=h1e[:N, :N], 
#                 g2e=g2e[:N, :N, N:, N:], 
#                 ecore=0., 
#                 iprint=0
#                 )

ket = driver.get_random_mps(tag="KET", bond_dim=BD, nroots=1)
bond_dims = [BD] * 7 + [2*BD] * 7
noises = [1e-4] * 7 + [1e-5] * 7 + [0]
thrds = [1e-5] * 7 + [1e-7] * 7
driver.dmrg(
            Hemb, ket, n_sweeps=20, bond_dims=bond_dims, 
            noises=noises, thrds=thrds, cutoff=1e-6, iprint=1,
            cached_contraction=True, tol=1e-6
            )


Sweep =    0 | Direction =  forward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      0.008 | E =     -10.3074867066 | DW = 3.88297e-06

Sweep =    1 | Direction = backward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      0.013 | E =     -10.3074867101 | DE = -3.51e-09 | DW = 8.06616e-08

Sweep =    2 | Direction =  forward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      0.018 | E =     -10.3074867092 | DE = 9.38e-10 | DW = 3.88280e-06

Sweep =    3 | Direction = backward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      0.023 | E =     -10.3074867116 | DE = -2.39e-09 | DW = 8.06480e-08

Sweep =    4 | Direction =  forward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      0.028 | E =     -10.3074867106 | DE = 9.82e-10 | DW = 3.88262e-06

Sweep =    5 | Direction = backward | 

-10.307487459503928

In [8]:
driver.orbital_reordering(h1e=[h1e[:N, :N], h1e[N:,N:]], g2e=[g2e[:N, :N, :N, :N], g2e[:N, :N, N:, N:], g2e[N:, N:, N:, N:]], method="gaopt")

array([3, 2, 0, 1])

In [50]:
driver.get_1pdm(ket)

array([[ 0.7244673 ,  0.1553166 , -0.30538592, -0.81194645],
       [ 0.1553166 ,  1.04230938, -0.91625405,  0.2850168 ],
       [-0.30538592, -0.91625405,  0.85438904, -0.08809999],
       [-0.81194645,  0.2850168 , -0.08809999,  1.37883428]])

In [53]:
driver.get_1pdm(ket)

array([[ 0.72451128,  0.15530942, -0.30535927, -0.8119761 ],
       [ 0.15530942,  1.04233331, -0.91626022,  0.28499324],
       [-0.30535927, -0.91626022,  0.85437543, -0.0881215 ],
       [-0.8119761 ,  0.28499324, -0.0881215 ,  1.37877997]])

In [71]:
idx

array([3, 2, 0, 1])

In [80]:
# reordered

idx = driver.orbital_reordering(h1e=h1e[:N, :N], g2e=g2e[:N, :N, N:, N:], method="gaopt")
Hemb = driver.get_qc_mpo(
                h1e=h1e[:N, :N], 
                g2e=g2e[:N, :N, N:, N:], 
                ecore=0., 
                iprint=0,
                reorder="gaopt"
                )

ket = driver.get_random_mps(tag="KET", bond_dim=BD, nroots=1)
bond_dims = [BD] * 7 + [2*BD] * 7
noises = [1e-4] * 7 + [1e-5] * 7 + [0]
thrds = [1e-5] * 7 + [1e-7] * 7
driver.dmrg(
            Hemb, ket, n_sweeps=20, bond_dims=bond_dims, 
            noises=noises, thrds=thrds, cutoff=1e-6, iprint=1,
            cached_contraction=True, tol=1e-6
            )


Sweep =    0 | Direction =  forward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      0.009 | E =      -4.3437769154 | DW = 2.01289e-07

Sweep =    1 | Direction = backward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      0.015 | E =      -4.3437769138 | DE = 1.54e-09 | DW = 1.90176e-07

Sweep =    2 | Direction =  forward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      0.021 | E =      -4.3437769163 | DE = -2.52e-09 | DW = 2.01293e-07

Sweep =    3 | Direction = backward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      0.027 | E =      -4.3437769148 | DE = 1.58e-09 | DW = 1.90175e-07

Sweep =    4 | Direction =  forward | Bond dimension = 1500 | Noise =  1.00e-04 | Dav threshold =  1.00e-05
Time elapsed =      0.033 | E =      -4.3437769173 | DE = -2.49e-09 | DW = 2.01297e-07

Sweep =    5 | Direction = backward | 

-4.343779512563217

In [ ]:
driver.re

array([2, 1, 0, 3, 6, 5, 4, 7])

In [82]:
eigval, eigvec = sop.diagonalize(nsites=N, Nparticle=(N//2,N//2), k=1)
eigvalr, eigvecr = sop_re.diagonalize(nsites=N, Nparticle=(N//2,N//2), k=1)
print(eigval - eigvalr)
print(eigval)

# make sure the h1e and g2e generated dmrg solver gives the same result as ED.

[-7.10542736e-15]
[-4.34377954]


In [83]:
idx = driver.reorder_idx

In [85]:
from gGA.operator.site_operator import S_m, S_p, S_z
S_m_ = sum(S_m(N, i) for i in range(N-1))
S_p_ = sum(S_p(N, i) for i in range(N-1))
S_z_ = sum(S_z(N, i) for i in range(N-1))

S2 = S_m_ * S_p_ + S_z_ * S_z_ + S_z_

v = S2.get_quspin_op(N, (N//2, N//2)).expt_value(eigvec)
nsites = N
h1e, g2e = S2.get_Hqc(nsites=nsites)
# h1e = 0.5 * (h1e + h1e.T)
ofd = 0.5 * (g2e[:nsites,:nsites,nsites:,nsites:] + g2e[nsites:,nsites:,:nsites,:nsites])
g2e[:nsites,:nsites,nsites:,nsites:] = ofd.copy()
g2e[nsites:,nsites:,:nsites,:nsites] = ofd.copy()
# S2op = driver.get_qc_mpo(
#     h1e=[h1e[:nsites, :nsites], h1e[nsites:,nsites:]], 
#     g2e=[g2e[:nsites, :nsites, :nsites, :nsites], g2e[:nsites, :nsites, nsites:, nsites:], g2e[nsites:, nsites:, nsites:, nsites:]], 
#     ecore=0., 
#     iprint=0
#     )
S2op = driver.get_qc_mpo(
    h1e=h1e[:nsites, :nsites], 
    g2e=g2e[:nsites, :nsites, nsites:, nsites:], 
    ecore=0., 
    iprint=0,
    reorder=idx
    )
vdmrg = np.array(driver.expectation(ket, S2op, ket).real).reshape(1,)
print(v, vdmrg)

[0.31959775] [0.31960349]


In [ ]:
driver.reorder_idx

None


In [ ]:
S2.op_list

In [ ]:
Operator.from_Hqc(h1e,g2e).op_list

In [ ]:
eigvs, _ = S2.diagonalize(N, (N//2,N//2))

eigvsr, _ = Operator.from_Hqc(h1e,g2e).diagonalize(N, (N//2,N//2))

print(eigvs, eigvsr)

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(h1e, cmap="bwr", vmax=1., vmin=-1.)
plt.show()

In [ ]:
from gGA.operator.site_operator import create_d, annihilate_d, create_u, annihilate_u, number_d, number_u
RDM = np.zeros((N, 2, N, 2))
for a in range(N):
    for s in range(2):
        for b in range(a, N):
            if a == b:
                start = s
            else:
                start = 0

            for s_ in range(start,2):
                if s == 0 and s_ == 0:
                    op = create_u(N, a) * annihilate_u(N, b)
                elif s == 1 and s_ == 1:
                    op = create_d(N, a) * annihilate_d(N, b)
                elif s == 0 and s_ == 1:
                    op = create_u(N, a) * annihilate_d(N, b)
                else:
                    op = create_d(N, a) * annihilate_u(N, b)
                
                v = op.get_quspin_op(N, (N//2,N//2)).expt_value(eigvec)
                RDM[a, s, b, s_] = v
                RDM[b, s_, a, s] = v.conj()

RDM_r = np.zeros((N, 2, N, 2))
for a in range(N):
    for s in range(2):
        for b in range(a, N):
            if a == b:
                start = s
            else:
                start = 0

            for s_ in range(start,2):
                if s == 0 and s_ == 0:
                    op = create_u(N, a) * annihilate_u(N, b)
                elif s == 1 and s_ == 1:
                    op = create_d(N, a) * annihilate_d(N, b)
                elif s == 0 and s_ == 1:
                    op = create_u(N, a) * annihilate_d(N, b)
                else:
                    op = create_d(N, a) * annihilate_u(N, b)
                
                v = op.get_quspin_op(N, (N//2,N//2)).expt_value(eigvecr)
                RDM_r[a, s, b, s_] = v
                RDM_r[b, s_, a, s] = v.conj()

In [ ]:
DMRG_RDM = driver.get_1pdm(ket)

In [ ]:
(RDM[:,0,:,0] - DMRG_RDM[0]).max()